# Qahf and Qh figure
- This script is used to visualize changes in anthropogenic heat flux (Qahf) and sensible heat flux (Qh) with different T_BUILD_MIN;
- Simulations: BASE, SENS;

In [31]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_sensitivity/urban_heat/'
result_df1 = pd.read_csv(output_dir + 'ahf.csv')
result_df2 = pd.read_csv(output_dir + 'qh.csv')
sites = ['AU-Pre','US-Bal','US-Wes']
file_date = ['2004-05-18', '2006-12-05', '2012-12-24']
start_date = ['2004-05-18T00:00:00', '2006-12-05T02:00:00','2012-12-24T00:00:00']
end_date = ['2004-05-25T23:30:00', '2006-12-12T01:30:00', '2012-12-31T23:30:00']
psites = ["AU-Preston","US-Baltimore","US-WestPhoenix"]
path = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/'
timestep = [336, 168, 336]

In [33]:
delta_list = []
for i in range(3):
    delta_site_list = []
    df_site = result_df1[result_df1['site']==sites[i]]
    for j in range(8):
        delta = df_site[df_site['index']==j]['urban_heat'].values-df_site[df_site['index']==8]['urban_heat'].values
        delta_df = pd.DataFrame({'delta': delta})
        delta_site_list.append(delta_df)
    delta_list.append(pd.concat(delta_site_list, axis=1, keys=[f'delta_{j}' for j in range(8)]))

In [34]:
delta_list2 = []
for i in range(3):
    delta_site_list2 = []
    df_site = result_df2[result_df2['site']==sites[i]]
    for j in range(8):
        delta = df_site[df_site['index']==j]['urban_heat'].values-df_site[df_site['index']==8]['urban_heat'].values
        delta_df = pd.DataFrame({'delta': delta})
        delta_site_list2.append(delta_df)
    delta_list2.append(pd.concat(delta_site_list2, axis=1, keys=[f'delta_{j}' for j in range(8)]))

In [35]:
y2 = []
for i in range(3):
    ob = xr.open_dataset(path + 'UrbanPlumber/Urban-PLUMBER_FullCollection_v1/' + psites[i] + '/timeseries/'+ psites[i] + '_clean_observations_v1.nc')
    ob_select_time = ob.sel(time=slice(start_date[i], end_date[i]))['Qh']
    y2.append(ob_select_time.mean().item())
    #print(ob_select_time.mean().values)
y2    

[-3.8215208053588867, 31.233341217041016, 21.711610794067383]

In [36]:
index = [0.8, 0.85, 0.9, 0.95, 1.05, 1.1, 1.15, 1.2]
index2 = [0.8, 0.85, 0.9, 0.95, 1, 1.05, 1.1, 1.15, 1.2]

# y1 is anthropogenic heat flux from Urban-PLUMBER metadata
y1 = [11, 5.5, 21.1]
#y2 = [-3.8215208, 25.139275, 21.71161]
labelfont = 6
linewidth = 0.1
linewidth2 = 0.5
labelcolor = '#6b6b6b'
lc0 = '#083550'
#lc1 = '#006bac'
lc1 = '#cb682c'
pad = 1
xwidth = 0.5
length = 2
alpha = 0.2
markersize = 1
handletextpad = 0.5
llc3 = [lc0, lc1, lc1]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(5.5, 2.7))
ax = axs.flatten()

padding = 4
for i in range(6):
    ax2 = ax[i].twinx()
    if i< 3:
        for j in range(timestep[i%3]):
            data = delta_list[i].iloc[j].values
            if j ==0:
                ax[i].plot(index, data, linewidth = linewidth, color = lc0, alpha = alpha, label = "SENS minus BASE")
            else:
                ax[i].plot(index, data, linewidth = linewidth, color = lc0, alpha = alpha)
            ax[i].set_title(f"({chr(97+i)}) {psites[i]}",fontsize=labelfont,color=labelcolor, pad = padding)
        ax[i].set_ylabel('ΔQahf', labelpad=5, rotation=0, ha='center', fontsize=labelfont,color=lc0)
        ax2.set_ylabel('Qahf', labelpad=5, rotation=0, ha='center', fontsize=labelfont,color=lc1)
        actual = result_df1[result_df1['site']==sites[i]].groupby('index')['urban_heat'].mean()
        actual_move = actual.drop(actual.index[-1])
        # Move the last element to the fourth position
        actual_after_move = pd.concat([actual_move.iloc[:4], pd.Series([actual.iloc[-1]], index=[4]), actual_move.iloc[4:]])
        ax2.plot(index2, actual_after_move, linewidth = linewidth2, marker = '.', markersize = markersize, color = lc1)
        ax2.axhline(y=y1[i], color=lc1, linestyle='--',linewidth = linewidth2)
        
    else:
        for j in range(timestep[i%3]):
            data = delta_list2[i-3].iloc[j].values
            ax[i].plot(index, data, linewidth = linewidth, color = lc0, alpha = alpha)
            ax[i].set_title(f"({chr(97+i)}) {psites[i-3]}",fontsize=labelfont,color=labelcolor, pad = padding)
        actual2 = result_df2[result_df2['site']==sites[i-3]].groupby('index')['urban_heat'].mean()
        actual_move2 = actual2.drop(actual2.index[-1])
        # Move the last element to the fourth position
        actual_after_move2 = pd.concat([actual_move2.iloc[:4], pd.Series([actual2.iloc[-1]], index=[4]), actual_move2.iloc[4:]])
        ax2.plot(index2, actual_after_move2, linewidth = linewidth2, marker = '.', 
                 markersize = markersize, label = "SENS simulations", color=lc1)  
        ax2.axhline(y=y2[i-3], color=lc1, linestyle='--', linewidth = linewidth2, label = "Observation")
        ax[i].set_ylabel('ΔQh', labelpad=5, rotation=0, ha='center', fontsize=labelfont,color=lc1)
        ax2.set_ylabel('Qh', labelpad=5, rotation=0, ha='center', fontsize=labelfont,color=lc1)
    
    #ax[i].set_title(f"({chr(97+i)}) {sites[i]}",fontsize=labelfont,color=labelcolor, pad = padding)   
    ax[i].tick_params(axis='y', which='both', left=True, right=False, labelleft=True, labelcolor=lc0, color=lc0,labelsize=labelfont,pad=pad, width = xwidth, length = length)
    ax[i].tick_params(axis='x', which='both', top=False, bottom=True, labelbottom=True, labelcolor=labelcolor,color=labelcolor,labelsize=labelfont,pad=pad, width = xwidth, length = length)
    ax[i].spines['top'].set_color(labelcolor)
    ax[i].spines['right'].set_color(lc1)
    ax[i].spines['bottom'].set_color(labelcolor)
    ax[i].spines['left'].set_color(lc0)
    ax[i].spines['top'].set_linewidth(xwidth)
    ax[i].spines['right'].set_linewidth(xwidth)
    ax[i].spines['bottom'].set_linewidth(xwidth)
    ax[i].spines['left'].set_linewidth(xwidth)
    ax[i].grid(True,linestyle='--',linewidth=0.5, alpha=0.5)   
    ax[i].set_xlim(0.8, 1.2)
    ax[i].set_ylim(-10, 10)
    #ax[i].set_xticks(index)
    ax[i].set_xticks([0.8, 0.85, 0.9, 0.95, 1, 1.05, 1.1, 1.15, 1.2])
    #ax[i].set_xticklabels([0.8, 0.85, 0.9, 0.95, 'BASE (1)', 1.05, 1.1, 1.15, 1.2])
    #ax[i].set_xticklabels([0.8, '', 0.9, '', 1, '', 1.1,  '', 1.2])
    ax[i].set_xticklabels(['-20%', '', '-10%', '', 'Baseline', '', '10%',  '', '20%'])
    #ax[i].yaxis.set_label_coords(0, 1.035)
    ax[i].set_xlabel('T_BUILDING_MIN perturbation', fontsize=labelfont, color=labelcolor, labelpad=1)
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    ax2.tick_params(axis='y', which='both', left=False, right=True, labelleft=False, labelright=True,labelcolor=lc1, color=lc1,labelsize=labelfont,pad=pad, width = xwidth, length = length)
    ax2.set_ylim(-10, 35)
    ax[i].yaxis.set_label_coords(0, 1.035)
    ax2.yaxis.set_label_coords(0.975, 1.11)

lines1, labels1 = ax[0].get_legend_handles_labels()  
lines2, labels2 = ax2.get_legend_handles_labels() 

for line in lines1:
    line.set_alpha(1) 
    line.set_linewidth(0.3)
combined_lines = lines1 + lines2
combined_labels = labels1 + labels2

legend1 = fig.legend(combined_lines, combined_labels, bbox_to_anchor=(0.5, -0.025), loc='lower center', frameon=False, handletextpad=handletextpad, fontsize=labelfont, ncol=3)
for i, text in enumerate(legend1.get_texts()):
    text.set_color(llc3[i])
    
plt.subplots_adjust(left=0.028, right=0.96, bottom=0.135, top=0.95, wspace=0.3, hspace= 0.45)
output_dir = '/mnt/iusers01/fatpou01/sees01/a16404ys/scratch/output_analysis/project2/sp_sensitivity/urban_heat/'
fig.savefig(output_dir + 'urban_heat.png', dpi=300)     
fig.savefig(output_dir + 'urban_heat.pdf', dpi=600)      

# Description

In [40]:
np.round(result_df1[result_df1['site']==sites[0]].groupby('index')['urban_heat'].mean(),1)

index
0    0.3
1    0.5
2    0.7
3    1.1
4    2.4
5    3.3
6    4.4
7    5.5
8    1.6
Name: urban_heat, dtype: float64

In [39]:
np.round(result_df1[result_df1['site']==sites[1]].groupby('index')['urban_heat'].mean(),1)

index
0    12.0
1    12.9
2    13.8
3    14.7
4    16.5
5    17.5
6    18.4
7    19.3
8    15.6
Name: urban_heat, dtype: float64

In [41]:
np.round(result_df1[result_df1['site']==sites[2]].groupby('index')['urban_heat'].mean(),1)

index
0    1.4
1    2.0
2    2.8
3    3.6
4    5.6
5    6.7
6    7.9
7    9.1
8    4.5
Name: urban_heat, dtype: float64

In [66]:
result_df2[result_df2['site']==sites[1]]

,time,lndgrid,urban_heat,hour,minute,index,site,date
3024,00:30,0,-12.964991,0,30,0,US-Mi1,2006-10-14 00:30:00
3025,01:00,0,-13.870537,1,0,0,US-Mi1,2006-10-14 01:00:00
3026,01:30,0,-12.584085,1,30,0,US-Mi1,2006-10-14 01:30:00
3027,02:00,0,-11.584373,2,0,0,US-Mi1,2006-10-14 02:00:00
3028,02:30,0,-13.230627,2,30,0,US-Mi1,2006-10-14 02:30:00
...,...,...,...,...,...,...,...,...
6043,22:00,0,35.920574,22,0,8,US-Mi1,2006-10-20 22:00:00
6044,22:30,0,34.503391,22,30,8,US-Mi1,2006-10-20 22:30:00
6045,23:00,0,26.983982,23,0,8,US-Mi1,2006-10-20 23:00:00
6046,23:30,0,24.028284,23,30,8,US-Mi1,2006-10-20 23:30:00
